In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np


import pandas as pd
import os


root = "/Users/zyxu/Documents/py/FM_weights"
os.chdir(root)


## load

In [2]:
loaded_tensor = torch.load(f'{root}/output/4sets_intermediate_outputs.pth')
print(loaded_tensor.shape)

torch.Size([13, 256, 197, 192])


In [3]:
from utils import intermdiates_to_pos_ctx
res = intermdiates_to_pos_ctx(loaded_tensor, normalize=False)
res.keys()

mu.shape torch.Size([13, 192])
expanded_mu_pos:  torch.Size([13, 197, 192])
expanded_mu_ctx:  torch.Size([13, 256, 192])
expanded_pos: torch.Size([13, 256, 197, 192]), expanded_ctx: torch.Size([13, 256, 197, 192]), expanded_mu_whole: torch.Size([13, 256, 197, 192])
resid: torch.Size([13, 256, 197, 192])
cvec: torch.Size([13, 256, 197, 192])


dict_keys(['mu', 'pos', 'ctx', 'resid', 'cvec'])

In [4]:
for key, val in res.items():
    print(key, val.shape)

mu torch.Size([13, 192])
pos torch.Size([13, 197, 192])
ctx torch.Size([13, 256, 192])
resid torch.Size([13, 256, 197, 192])
cvec torch.Size([13, 256, 197, 192])


## look into function

In [31]:
pos = res['pos'].detach().numpy()
global_mean = res['mu'] .detach().numpy()
ctx = res['ctx'].detach().numpy()
resid = res['resid'].detach().numpy()
cvec = res['cvec'].detach().numpy()
L, T, D = pos.shape
L, B, T, D = cvec.shape
nseq = B

In [32]:
l = 0
p = pos[l]  # [T, C]
c = cvec[l] # [B, T, C]
g = global_mean[l]

p_full = np.stack([p for _ in range(nseq)])
p_full.shape

(256, 197, 192)

In [33]:
m = p_full + c + g
m.shape

(256, 197, 192)

In [34]:
g.shape

(192,)

## debug

## whole func

In [5]:
from utils.metrics import ranks_and_explained_ratios_and_relative_norm_customize
ranks = ranks_and_explained_ratios_and_relative_norm_customize(pos = res['pos'].detach().numpy(), 
                                                     cvec = res['cvec'].detach().numpy(), 
                                                     global_mean=res['mu'].detach().numpy())

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:05<00:00,  2.32it/s]


In [6]:
# pos = res['pos'].detach().numpy()
# cvec = res['cvec'].detach().numpy()
# global_mean=res['mu'].detach().numpy()
# l = 0
# B = 256

# p = pos[l] # [T, C]
# c = cvec[l] # [B, T, C]
# g = global_mean[l]
# p_full = np.stack([p for _ in range(B)])
# p_full.shape

# m = p_full + c + g

In [7]:
ranks

array([[14.        ,  5.94665623,  0.74288195,  0.44029891],
       [13.        ,  6.45518303,  0.7479775 ,  0.26936308],
       [14.        ,  6.71914434,  0.76990777,  0.33433321],
       [12.        ,  4.75218105,  0.69340789,  0.35889575],
       [13.        ,  4.15983725,  0.68868089,  0.33732805],
       [13.        ,  3.88320684,  0.68567467,  0.40138921],
       [13.        ,  2.76894569,  0.59528911,  0.32614145],
       [13.        ,  1.6177597 ,  0.63269818,  0.60998422],
       [11.        ,  1.0009979 ,  0.92308134,  0.9215911 ],
       [11.        ,  1.00108099,  0.90798295,  0.91557604],
       [11.        ,  1.00107646,  0.89735132,  0.91477895],
       [11.        ,  1.00111032,  0.88898629,  0.91686368],
       [10.        ,  1.00113273,  0.88381577,  0.93325359]])

Table 1: ScreeNot

In [8]:
index = ["ScreeNot", "stable_rank", "explained_ratio", "relative_norm"]
d = []
columns = [f"Layer {i}" for i in range(13)]
for id,metric in enumerate(["ScreeNot", "stable_rank", "explained_ratio", "relative_norm"]):
    c = ranks[:, id]
    d.append(c)

In [9]:
pd.DataFrame(np.array(d),index = index, columns=columns).round(3)

,Layer 0,Layer 1,Layer 2,Layer 3,Layer 4,Layer 5,Layer 6,Layer 7,Layer 8,Layer 9,Layer 10,Layer 11,Layer 12
ScreeNot,14.000,13.000,14.000,12.000,13.000,13.000,13.000,13.000,11.000,11.000,11.000,11.000,10.000
stable_rank,5.947,6.455,6.719,4.752,4.160,3.883,2.769,1.618,1.001,1.001,1.001,1.001,1.001
explained_ratio,0.743,0.748,0.770,0.693,0.689,0.686,0.595,0.633,0.923,0.908,0.897,0.889,0.884
relative_norm,0.440,0.269,0.334,0.359,0.337,0.401,0.326,0.610,0.922,0.916,0.915,0.917,0.933
